In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf


In [2]:
#  Import and read the charity_data.csv.
import pandas as pd 
motor_df = pd.read_csv('Resources/Motor_Vehicle_Crashes_-_Vehicle_Information__Three_Year_Window.csv')
motor_df.head()

,Year,Case Vehicle ID,Vehicle Body Type,Registration Class,Action Prior to Accident,Type / Axles of Truck or Bus,Direction of Travel,Fuel Type,Vehicle Year,State of Registration,Number of Occupants,Engine Cylinders,Vehicle Make,Contributing Factor 1,Contributing Factor 1 Description,Contributing Factor 2,Contributing Factor 2 Description,Event Type,Partial VIN
0,2018,15232120,VAN TRUCK,AGRICULTURAL TRUCK,Parked,Not Entered,North,Gas,2015.0,NY,0.0,4.0,NISSA,HUMAN,Not Entered,HUMAN,Not Entered,Not Entered,3N6CM0KN2FK713460
1,2018,15232535,SUBURBAN,PASSENGER OR SUBURBAN,Making U Turn,Not Entered,South,Gas,2015.0,NY,1.0,6.0,HONDA,HUMAN,Not Entered,HUMAN,Unknown,Not Entered,5J6TF2H55FL001556
2,2018,15232536,SEDAN,Not Entered,Going Straight Ahead,Not Entered,North,Not Entered,NaN,PA,1.0,NaN,NaN,HUMAN,Not Entered,HUMAN,Unknown,Not Entered,NaN
3,2018,15232557,4 DOOR SEDAN,PASSENGER OR SUBURBAN,Parked,Not Entered,South,Gas,2013.0,NY,5.0,4.0,NISSA,HUMAN,Passenger Distraction,VEHICLE,Other*,Not Entered,1N4AL3AP8DC229156
4,2018,15232556,SUBURBAN,PASSENGER OR SUBURBAN,Going Straight Ahead,Not Entered,South,Gas,2009.0,NY,2.0,8.0,CADIL,HUMAN,Not Applicable,HUMAN,Not Applicable,Not Entered,3GYFK22209G213337


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME','Contributing Factor 1 Description',
# 'Contributing Factor 2 Description'.

motor_df = motor_df.drop(['Case Vehicle ID', 'Partial VIN', 'Contributing Factor 1 Description',
                          'Contributing Factor 2 Description'], axis=1)

In [3]:
# Dropping rows with no value

motor_df = motor_df.dropna()

In [4]:
# Determine the number of unique values in each column.

motor_df.nunique()

Year                                       2
Case Vehicle ID                      1252633
Vehicle Body Type                         51
Registration Class                        61
Action Prior to Accident                  21
Type / Axles of Truck or Bus               5
Direction of Travel                       10
Fuel Type                                  8
Vehicle Year                              95
State of Registration                     14
Number of Occupants                       64
Engine Cylinders                          14
Vehicle Make                            1210
Contributing Factor 1                      3
Contributing Factor 1 Description         58
Contributing Factor 2                      3
Contributing Factor 2 Description         57
Event Type                                34
Partial VIN                          1157572
dtype: int64

In [5]:
# Looking at the Vehicle Make column

motor_count = motor_df['Vehicle Make'].value_counts()

In [6]:
# Determine which values to replace if counts are less than 5000

replace_Vehicle = list(motor_count[motor_count < 25000].index)
replace_Vehicle
# Replace in dataframe
for car in replace_Vehicle:
    motor_df["Vehicle Make"] = motor_df["Vehicle Make"].replace(car,"Other")
    
# Check to make sure binning was successful
motor_df['Vehicle Make'].value_counts()

Other    291615
TOYOT    177389
HONDA    147435
FORD     123620
CHEVR    121472
NISSA    110520
JEEP      60809
HYUND     54557
DODGE     44049
SUBAR     39699
BMW       27585
KIA       27095
ME/BE     26789
Name: Vehicle Make, dtype: int64

In [7]:
# Get Names for indexs for which Contributing Factor  1 Description 1 is Not Entered

not_entered = motor_df[motor_df['Contributing Factor 1 Description'] == 'Not Entered'].index

In [8]:
# Drop these rows

motor_df.drop(not_entered, inplace = True)

In [9]:
# Get Names for indexs for which Contributing Factor  1 Description 1 is Not applicable

not_app = motor_df[motor_df['Contributing Factor 1 Description'] == 'Not Applicable'].index

In [10]:
# Drop these rows

motor_df.drop(not_app, inplace = True)

In [11]:
# Get Names for index for which Contributing Factor 1 Description is Unknown

unknown = motor_df[motor_df['Contributing Factor 1 Description'] == 'Unknown'].index

In [12]:
# Drop these rows

motor_df.drop(unknown, inplace = True)

In [13]:
motor_df.head()

,Year,Case Vehicle ID,Vehicle Body Type,Registration Class,Action Prior to Accident,Type / Axles of Truck or Bus,Direction of Travel,Fuel Type,Vehicle Year,State of Registration,Number of Occupants,Engine Cylinders,Vehicle Make,Contributing Factor 1,Contributing Factor 1 Description,Contributing Factor 2,Contributing Factor 2 Description,Event Type,Partial VIN
3,2018,15232557,4 DOOR SEDAN,PASSENGER OR SUBURBAN,Parked,Not Entered,South,Gas,2013.0,NY,5.0,4.0,NISSA,HUMAN,Passenger Distraction,VEHICLE,Other*,Not Entered,1N4AL3AP8DC229156
20,2018,15233368,4 DOOR SEDAN,PASSENGER OR SUBURBAN,Going Straight Ahead,Not Entered,West,Gas,2007.0,NY,2.0,6.0,Other,HUMAN,Reaction to Other Uninvolved Vehicle,ENVMT,Pavement Slippery,Not Applicable,1G4HD57257U210376
21,2018,15233376,4 DOOR SEDAN,PASSENGER OR SUBURBAN,Going Straight Ahead,Not Entered,North,Gas,2013.0,NY,1.0,4.0,DODGE,ENVMT,Animal's Action,ENVMT,Pavement Slippery,Not Applicable,1C3CDZAB5DN710899
31,2018,15233845,4 DOOR SEDAN,PASSENGER OR SUBURBAN,Going Straight Ahead,Not Entered,South,Gas,2014.0,NY,1.0,4.0,FORD,HUMAN,Unsafe Speed,ENVMT,Pavement Slippery,Not Applicable,3FADP4BJ4EM215708
34,2018,15233850,SUBURBAN,PASSENGER OR SUBURBAN,Backing,Not Entered,East,Gas,2006.0,NY,3.0,6.0,HYUND,HUMAN,Backing Unsafely,HUMAN,Driver Inattention/Distraction*,Not Applicable,KM8JN72D76U425890


In [14]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME','Contributing Factor 1 Description',
# 'Contributing Factor 2 Description'.

motor_df = motor_df.drop(['Case Vehicle ID', 'Partial VIN', 'Contributing Factor 2'], axis=1)

In [15]:
# The number of contributing factor to vehicle in NYC

len(motor_df['Contributing Factor 1 Description'].value_counts())


55

In [16]:
# Determining the dependent factor for the results of these contributing factors

events = motor_df['Contributing Factor 1'].value_counts()
events

HUMAN      356999
ENVMT       49990
VEHICLE     11982
Name: Contributing Factor 1, dtype: int64

In [17]:
# Determine wheather the accident is caused by humans or another factor

replace_events = list(events[events < 60000].index)
replace_events
# Replace in dataframe
for event in replace_events:
    motor_df["Contributing Factor 1"] = motor_df["Contributing Factor 1"].replace(event, "Other")
    
# Check to make sure binning was successful
motor_df['Contributing Factor 1'].value_counts()


HUMAN    356999
Other     61972
Name: Contributing Factor 1, dtype: int64

In [19]:
# Recode Contributing Factor 1 to reflect integers instead of strings and define dependent variable

def Classification(X):
    if X == 'HUMAN':
        return 0
    else :
        return 1
    
motor_df['Contributing Factor 1'] = motor_df['Contributing Factor 1'].apply(Classification)
motor_df.head()

,Year,Vehicle Body Type,Registration Class,Action Prior to Accident,Type / Axles of Truck or Bus,Direction of Travel,Fuel Type,Vehicle Year,State of Registration,Number of Occupants,Engine Cylinders,Vehicle Make,Contributing Factor 1,Contributing Factor 1 Description,Contributing Factor 2 Description,Event Type
3,2018,4 DOOR SEDAN,PASSENGER OR SUBURBAN,Parked,Not Entered,South,Gas,2013.0,NY,5.0,4.0,NISSA,0,Passenger Distraction,Other*,Not Entered
20,2018,4 DOOR SEDAN,PASSENGER OR SUBURBAN,Going Straight Ahead,Not Entered,West,Gas,2007.0,NY,2.0,6.0,Other,0,Reaction to Other Uninvolved Vehicle,Pavement Slippery,Not Applicable
21,2018,4 DOOR SEDAN,PASSENGER OR SUBURBAN,Going Straight Ahead,Not Entered,North,Gas,2013.0,NY,1.0,4.0,DODGE,1,Animal's Action,Pavement Slippery,Not Applicable
31,2018,4 DOOR SEDAN,PASSENGER OR SUBURBAN,Going Straight Ahead,Not Entered,South,Gas,2014.0,NY,1.0,4.0,FORD,0,Unsafe Speed,Pavement Slippery,Not Applicable
34,2018,SUBURBAN,PASSENGER OR SUBURBAN,Backing,Not Entered,East,Gas,2006.0,NY,3.0,6.0,HYUND,0,Backing Unsafely,Driver Inattention/Distraction*,Not Applicable


In [20]:
# Generate our categorical variable lists
motor_cat = motor_df.dtypes[motor_df.dtypes == "object"].index.tolist()
motor_cat

['Vehicle Body Type',
 'Registration Class',
 'Action Prior to Accident',
 'Type / Axles of Truck or Bus',
 'Direction of Travel',
 'Fuel Type',
 'State of Registration',
 'Vehicle Make',
 'Contributing Factor 1 Description',
 'Contributing Factor 2 Description',
 'Event Type']

In [21]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list

encode_df = pd.DataFrame(enc.fit_transform(motor_df[motor_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(motor_cat)
encode_df.head()

,Vehicle Body Type_2 DOOR SEDAN,Vehicle Body Type_4 DOOR SEDAN,Vehicle Body Type_ALL TERRAIN VEHICLE,Vehicle Body Type_AMBULANCE,Vehicle Body Type_BUS (OMNIBUS),Vehicle Body Type_CEMENT MIXER,Vehicle Body Type_CONSTRUCTION EQUIPMENT,Vehicle Body Type_CONVERTIBLE,Vehicle Body Type_CUSTOM,Vehicle Body Type_DELIVERY TRUCK,...,"Event Type_Other*, Non-Collision","Event Type_Overturned, Non-Collision","Event Type_Pedestrian, Collision With","Event Type_Railroad Train, Collision With","Event Type_Ran Off Roadway Only, Non-Collision","Event Type_Sign Post, Collision With Fixed Object","Event Type_Snow Embankment, Collision With Fixed Object","Event Type_Submersion, Non-Collision","Event Type_Tree, Collision With Fixed Object",Event Type_Unknown
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# Merge one-hot encoded features and drop the originals

motor_df = motor_df.merge(encode_df, left_index=True,right_index=True)
motor_df = motor_df.drop(motor_cat, 1)
motor_df.head()

,Year,Vehicle Year,Number of Occupants,Engine Cylinders,Contributing Factor 1,Vehicle Body Type_2 DOOR SEDAN,Vehicle Body Type_4 DOOR SEDAN,Vehicle Body Type_ALL TERRAIN VEHICLE,Vehicle Body Type_AMBULANCE,Vehicle Body Type_BUS (OMNIBUS),...,"Event Type_Other*, Non-Collision","Event Type_Overturned, Non-Collision","Event Type_Pedestrian, Collision With","Event Type_Railroad Train, Collision With","Event Type_Ran Off Roadway Only, Non-Collision","Event Type_Sign Post, Collision With Fixed Object","Event Type_Snow Embankment, Collision With Fixed Object","Event Type_Submersion, Non-Collision","Event Type_Tree, Collision With Fixed Object",Event Type_Unknown
3,2018,2013.0,5.0,4.0,0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,2018,2007.0,2.0,6.0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,2018,2013.0,1.0,4.0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,2018,2014.0,1.0,4.0,0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,2018,2006.0,3.0,6.0,0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
motor_df['Contributing Factor 1']

3         0
20        0
21        1
31        0
34        0
         ..
418955    0
418959    0
418961    0
418963    0
418968    0
Name: Contributing Factor 1, Length: 54304, dtype: int64

In [28]:
# Split our preprocessed data into our features and target arrays

y = motor_df['Contributing Factor 1'].values
X = motor_df.drop("Contributing Factor 1", axis = 1).values

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [29]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation  ="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation ="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation ="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 2496      
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 2,547
Trainable params: 2,547
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [32]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
1273/1273 [==============================] - 5s 2ms/step - loss: 1.5846 - accuracy: 0.9498
Epoch 2/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.1197 - accuracy: 0.9744
Epoch 3/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.1198 - accuracy: 0.9744
Epoch 4/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.1202 - accuracy: 0.9744
Epoch 5/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.1203 - accuracy: 0.9744
Epoch 6/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.1208 - accuracy: 0.9744
Epoch 7/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.1209 - accuracy: 0.9744
Epoch 8/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.1206 - accuracy: 0.9744
Epoch 9/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.1215 - accuracy: 0.9744
Epoch 10/100
1273/1273 [==============================] - 2s 2ms/step - l

1273/1273 [==============================] - 2s 2ms/step - loss: 0.1190 - accuracy: 0.9744
Epoch 79/100
1273/1273 [==============================] - 2s 1ms/step - loss: 0.1187 - accuracy: 0.9744
Epoch 80/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.1186 - accuracy: 0.9744
Epoch 81/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.1192 - accuracy: 0.9744
Epoch 82/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.1190 - accuracy: 0.9744
Epoch 83/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.1187 - accuracy: 0.9744
Epoch 84/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.1188 - accuracy: 0.9744
Epoch 85/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.1186 - accuracy: 0.9744
Epoch 86/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.1188 - accuracy: 0.9744
Epoch 87/100
1273/1273 [==============================] - 2s 2ms/step - loss:

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

425/425 - 1s - loss: 0.1193 - accuracy: 0.9744
Loss: 0.11930986493825912, Accuracy: 0.9744402170181274
